In [2]:
import os

products_collection_name = 'amazon-products'
reviews_collection_name = 'amazon-reviews'
reviews_path = os.path.join(os.getcwd(), reviews_collection_name)
products_path = os.path.join(os.getcwd(), products_collection_name)
create_collection(reviews_path, reviews_collection_name)
create_collection(products_path, products_collection_name)

In [3]:
import html
    
def product_json(filename):
    list_of_dictionaries = []
    for item in open(filename):
        dict = {}
        line = eval('(' + item + ')')
        if 'title' in line:
            dict.update({'title': html.unescape(line['title'])})
        else:
            continue
        if 'description' in line:
            dict.update({'description': html.unescape(line['description'])})
        if 'price' in line:
            dict.update({'price': float('{:.2f}'.format(line['price']))})
        dict.update({'asin': line['asin']})
        list_of_dictionaries.append(dict)
    return list_of_dictionaries

def review_json(filename, products):
    list_of_dictionaries = []
    product_asins = [x['asin'] for x in products]
    for item in open(filename):
        dict = {}
        line = eval('(' + item + ')')
        if line['asin'] in product_asins:
            dict.update({'reviewText': line['reviewText']})
            dict.update({'overall': int(line['overall'])})
            dict.update({'summary': line['summary']})
            dict.update({'asin': line['asin']})
            list_of_dictionaries.append(dict)
    return list_of_dictionaries

In [4]:
import pysolr
def index_to_solr(data, port=8983, collection_name=''):
    solr = pysolr.Solr(f'http://localhost:{port}/solr/{collection_name}')
    solr.add(data, commit=True)

def index_document(product_filename, review_filename):
    products = product_json(product_filename)
    reviews = review_json(review_filename, products)
    index_to_solr(products, collection_name=products_collection_name)
    index_to_solr(reviews, collection_name=reviews_collection_name)

# Documentation
In order to run the application, you must first do `pip install flask-bootstrap`.

Running the code will execute (in order):
    *index the .txt files into two json arrays named `products` and `reviews`. 
    *index the two json arrays to solr.
    
-------
# Assumptions
`flask & jinja` libraries are already installed